In [ ]:
import numpy as np
import open3d as o3d
import copy
import pickle
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import time
from scipy.spatial import cKDTree
from joblib import Parallel, delayed

from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.IFSelect import IFSelect_RetDone
from OCC.Core.TopAbs import TopAbs_WIRE, TopAbs_EDGE, TopAbs_VERTEX
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopoDS import topods
from OCC.Core.BRep import BRep_Tool
from OCC.Core.BRepAdaptor import BRepAdaptor_Curve
from OCC.Core.GeomAbs import GeomAbs_Circle, GeomAbs_Ellipse, GeomAbs_Line, GeomAbs_BSplineCurve, GeomAbs_Hyperbola, GeomAbs_Parabola
from OCC.Core.TopAbs import TopAbs_WIRE, TopAbs_EDGE, TopAbs_VERTEX
from OCC.Display.SimpleGui import init_display
from OCC.Core.Quantity import Quantity_Color, Quantity_TOC_RGB
from OCC.Core.gp import gp_Pnt
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeVertex
from PyQt5 import QtWidgets
from OCC.Core.GCPnts import GCPnts_UniformDeflection
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop

In [ ]:
# Read processed point cloud data

point_dictionary_path = '/home/chris/Code/PointClouds/data/hull_pcd/morten_plate/mapped_hull_points_dictionary.pkl'

hull_centers_dictionary = {}

with open(point_dictionary_path, 'rb') as f:
    hull_points_dictionary = pickle.load(f)

hull_pcd = o3d.geometry.PointCloud()

for label, points in hull_points_dictionary.items():
    hull_pcd.points.extend(o3d.utility.Vector3dVector(points))
    center = np.mean(points, axis=0)
    hull_centers_dictionary[label] = center
    
hull_pcd.paint_uniform_color([0.1, 1, 0.1])

hull_centers_pcd = o3d.geometry.PointCloud()
hull_centers_points = list(hull_centers_dictionary.values())
hull_centers_pcd.points = o3d.utility.Vector3dVector(np.array(hull_centers_points))
hull_centers_pcd.paint_uniform_color([1, 0, 0])  # Color centers red

print("Point cloud data has been loaded successfully.")

In [ ]:
# Read cad model data (STEP file)

step_file_path = '/home/chris/Code/PointClouds/data/other_files/MortenPartSTEPVersion.STEP'

def load_step_file(file_path):
    step_reader = STEPControl_Reader()
    status = step_reader.ReadFile(file_path)
    if status == IFSelect_RetDone:
        step_reader.TransferRoots()
        shape = step_reader.OneShape()
        return shape
    else:
        raise Exception("Error: Cannot read STEP file.")

shape = load_step_file(step_file_path)

print('Step file has been loaded successfully.')

In [ ]:
# Extract edges at the top Z-coordinate
def extract_top_edges(shape, deflection=0.1):
    edges = []
    max_z = -float('inf')
    edge_z_coordinates = []

    # First pass to collect the highest Z-coordinate of each edge
    exp_edge = TopExp_Explorer(shape, TopAbs_EDGE)
    while exp_edge.More():
        edge = topods.Edge(exp_edge.Current())
        edge_max_z = -float('inf')
        exp_vertex = TopExp_Explorer(edge, TopAbs_VERTEX)
        vertices = []
        while exp_vertex.More():
            vertex = topods.Vertex(exp_vertex.Current())
            point = BRep_Tool.Pnt(vertex)
            vertices.append(point)
            if point.Z() > edge_max_z:
                edge_max_z = point.Z()
            exp_vertex.Next()
        edge_z_coordinates.append((edge, edge_max_z, vertices))
        if edge_max_z > max_z:
            max_z = edge_max_z
        exp_edge.Next()

    #print(f"Total number of edges: {len(edge_z_coordinates)}")
    #print(f"Maximum Z-coordinate found: {max_z}")

    # Collect edges on the top plane
    top_edges = []
    point_cloud = []
    for edge, z, vertices in edge_z_coordinates:
        if all(abs(vertex.Z() - max_z) < 1e-3 for vertex in vertices):  # Ensure all vertices are at the top surface
            top_edges.append(edge)
            #print(f"Top edge added with highest vertex Z: {z}")

            # Sample points along the edge
            curve = BRepAdaptor_Curve(edge)
            u_min, u_max = curve.FirstParameter(), curve.LastParameter()

            # Calculate the length of the edge
            linear_props = GProp_GProps()
            brepgprop.LinearProperties(edge, linear_props)
            length = linear_props.Mass()

            # Calculate the number of samples based on the length and the desired point density
            num_samples = int(length / deflection)

            for i in range(num_samples):
                u = u_min + i * (u_max - u_min) / (num_samples - 1)
                pnt = curve.Value(u)
                point_cloud.append((pnt.X(), pnt.Y(), pnt.Z()))  # Store the coordinates as a tuple

    print(f"Total number of top view edges: {len(top_edges)}")
    print(f"Total number of points in the cad point cloud: {len(point_cloud)}")
    return top_edges, point_cloud

# Visualize the top edges
def visualize_top_edges(top_edges):
    display, start_display, add_menu, add_function_to_menu = init_display()

    # Assign the color red to each top edge
    red_color = Quantity_Color(1.0, 0.0, 0.0, Quantity_TOC_RGB)
    
    # Display each top edge with the red color
    for edge in top_edges:
        display.DisplayShape(edge, update=True, color=red_color)

    start_display()


# Main execution
print('Generating top view edges and sampling to create the associated point cloud...')
top_edges, cad_points = extract_top_edges(shape)
# if top_edges:
#     visualize_top_edges(top_edges)
# else:
#     print("Top edges not found.")

cad_pcd = o3d.geometry.PointCloud()
cad_pcd.points = o3d.utility.Vector3dVector(np.asarray(cad_points))
o3d.visualization.draw_geometries([cad_pcd])

print('Top view cad point cloud created.')

In [ ]:
# Apply clustering to the cad_pcd

eps = 0.50
min_points = 10

cad_shapes_labels = np.array(cad_pcd.cluster_dbscan(eps=eps, min_points=min_points, print_progress=True))

max_label = cad_shapes_labels.max()
print(f"Cad point cloud has {max_label + 1} clusters")
base_cmap = plt.get_cmap("tab20")
color_cycle = [base_cmap(i % 20) for i in range(max_label + 1)]
colors = np.array(color_cycle)[cad_shapes_labels]
colors[cad_shapes_labels<0] = 0
cad_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

cad_points_dictionary = {}
cad_centers_dictionary = {}

# Extract the points and calculate the centers for each cluster
for label in np.unique(cad_shapes_labels):
    if label != -1:  # Ignore noise
        cluster_points = np.asarray(cad_pcd.points)[cad_shapes_labels == label]
        cad_points_dictionary[label] = cluster_points
        cad_centers_dictionary[label] = np.mean(cluster_points, axis=0)

cad_centers_pcd = o3d.geometry.PointCloud()
cad_centers_points = list(cad_centers_dictionary.values())
cad_centers_pcd.points = o3d.utility.Vector3dVector(np.array(cad_centers_points))
cad_centers_pcd.paint_uniform_color([0, 1, 0])  # Color centers blue

o3d.visualization.draw_geometries([cad_pcd])

In [ ]:
#Center based alignment: Align point clouds through ICP based on the lowest RMSE (only use centers of shapes)

cad_centers_pcd_center = np.mean(np.asarray(cad_centers_pcd.points), axis=0)
hull_centers_pcd_center = np.mean(np.asarray(hull_centers_pcd.points), axis=0)

# Compute the translation vector
translation = cad_centers_pcd_center - hull_centers_pcd_center

# Translate the hull_pcd to align the centers
hull_centers_pcd.points = o3d.utility.Vector3dVector(np.asarray(hull_centers_pcd.points) + translation)
hull_pcd.points = o3d.utility.Vector3dVector(np.asarray(hull_pcd.points) + translation)

o3d.visualization.draw_geometries([cad_centers_pcd, hull_centers_pcd])

In [ ]:
cad_centers_points = np.asarray(cad_centers_pcd.points)[:, :2]
hull_centers_points = np.asarray(hull_centers_pcd.points)[:, :2]

def get_rotation_matrix(angle):
    return np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])

def apply_rotation(centered_clouds, angle):
    rotation_matrix = get_rotation_matrix(angle)
    rotated_clouds = np.einsum('ij,nkj->nki', rotation_matrix, centered_clouds)
    return rotated_clouds

def calculate_distances_optimized(rotated_clouds, cloud_B):
    tree = cKDTree(cloud_B)
    distances, _ = tree.query(rotated_clouds.reshape(-1, 2))
    return distances.reshape(rotated_clouds.shape[0], rotated_clouds.shape[1])

def process_angle(angle, centered_clouds, cloud_B, centering_vectors):
    rotated_clouds = apply_rotation(centered_clouds, angle)
    uncentered_rotated_clouds = rotated_clouds + centering_vectors[:, np.newaxis, :]
    distances = calculate_distances_optimized(uncentered_rotated_clouds, cloud_B)
    avg_shortest_distances = np.mean(distances, axis=1)
    min_distance_idx = np.argmin(avg_shortest_distances)
    return avg_shortest_distances[min_distance_idx], angle, min_distance_idx

def ransac_match_clouds(cloud_A, cloud_B):
    print(cloud_A.shape)
    print(cloud_B.shape)
    best_transformation = None
    min_avg_distance = float('inf')
    angles = np.arange(-np.pi, np.pi, np.pi/720)

    cloud_A_indices = np.arange(cloud_A.shape[0])
    cloud_B_indices = np.arange(cloud_B.shape[0])
    combinations = np.array(np.meshgrid(cloud_A_indices, cloud_B_indices)).T.reshape(-1, 2)
    shuffled_combinations = np.random.permutation(combinations)
    translations = cloud_B[shuffled_combinations[:, 1]] - cloud_A[shuffled_combinations[:, 0]]

    #print("Translations shape:", translations.shape)

    translated_clouds = cloud_A[np.newaxis, :, :] + translations[:, np.newaxis, :]
    centering_vectors = cloud_B[shuffled_combinations[:, 1]]
    centered_clouds = translated_clouds - centering_vectors[:, np.newaxis, :]

    #print("Centered clouds shape:", centered_clouds.shape)

    results = Parallel(n_jobs=-1)(delayed(process_angle)(angle, centered_clouds, cloud_B, centering_vectors) for angle in tqdm(angles))

    for avg_distance, angle, idx in results:
        if avg_distance < min_avg_distance:
            min_avg_distance = avg_distance
            best_rotation_matrix = get_rotation_matrix(angle)
            best_translation_vector = translations[idx]
            best_centering_vector = centering_vectors[idx]
            best_transformation = np.eye(3)
            best_rotation_matrix_aug = np.eye(3)
            best_rotation_matrix_aug[:2, :2] = best_rotation_matrix
            
            # Augment the translation vectors to 3x3
            T1 = np.eye(3)
            T1[:2, 2] = best_translation_vector
            
            T2 = np.eye(3)
            T2[:2, 2] = best_centering_vector
            
            T3 = np.eye(3)
            T3[:2, 2] = -best_centering_vector
            
            # Combine the transformations
            best_transformation = T1 @ T2 @ best_rotation_matrix_aug @ T3

    #print("Best rotation matrix:", best_rotation_matrix)
    #print("Best translation vector:", best_translation_vector)
    print("Minimum average distance:", min_avg_distance)

    return best_transformation, min_avg_distance

best_transformation, min_avg_distance = ransac_match_clouds(cad_centers_points, hull_centers_points)

In [ ]:
best_transformation_3d = np.eye(4)
best_transformation_3d[:2, :2] = best_transformation[:2, :2]
best_transformation_3d[:2, 3] = best_transformation[:2, 2]

#print("Best 3D transformation matrix: \n", best_transformation_3d)

# Apply the transformation to a copy of the hull_centers_pcd for visualization
cad_centers_pcd_transformed = cad_centers_pcd.transform(best_transformation_3d)

# Visualize the final aligned point clouds
o3d.visualization.draw_geometries([hull_centers_pcd, cad_centers_pcd_transformed])

In [ ]:
best_transformation_3d = np.eye(4)
best_transformation_3d[:2, :2] = best_transformation[:2, :2]
best_transformation_3d[:2, 3] = best_transformation[:2, 2]

#print("Best 3D transformation matrix: \n", best_transformation_3d)

# Apply the transformation to a copy of the hull_centers_pcd for visualization
cad_pcd_transformed = cad_pcd.transform(best_transformation_3d)

# Visualize the final aligned point clouds
o3d.visualization.draw_geometries([hull_pcd, cad_pcd_transformed])